In [ ]:
import os
import pickle
from tqdm import tqdm
import numpy as np
import numpy.random as rand

from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd

import snorkel
from snorkel.augmentation import transformation_function
from snorkel.augmentation import RandomPolicy, MeanFieldPolicy
from snorkel.augmentation import TFApplier, PandasTFApplier

from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier

# Load Data

In [ ]:
# "../../../w3c-emails/emails.pkl"
with open("../../../../w3c-emails/emails.pkl", "rb") as handle:
    emails = [e for e in tqdm(pickle.load(handle))]

# rng = list(range(len(emails)))
for i, e in enumerate(emails):
    e.index = i
    e.org = e.sender.org

orgs = [e.org for e in emails]
# senders = [e.sender for e in emails]

In [ ]:
# "emails_token_ids.pkl"
with open("1000_emails_token_ids.pkl", "rb") as handle:
    email_ids = pickle.load(handle)

# Put into DF $\rightarrow$ AS COMBINATIONS

In [ ]:
names = "index", "id", "org", "body_raw"

# names2 = tuple(n + "_2" for n in names)

def by_author(emails):
    d = {}
    
    for e in emails:
        if not e.sender in d:
            d[e.sender] = []
        d[e.sender].append(e)
    return d

def by_org(emails):
    d = {}
    
    for e in emails:
        if not e.org in d:
            d[e.org] = []
        d[e.org].append(e)
    return d

# by_auths = by_author(emails)
by_orgs = by_org(emails)

        
def objs_to_frame(emails, min_size=1):
    rows = []
    for s, e_ls in by_orgs.items():
        if len(e_ls) < min_size:
            continue
        e_ls2 = rand.permutation(e_ls)
        for e1, e2 in zip(e_ls, e_ls2):
            row = {n: str(getattr(e1, n)) for n in names}
            row.update({n + "_2": str(getattr(e2, n)) for n in names})
            rows.append(row)
    return pd.DataFrame(rows)

mail_frame = objs_to_frame(emails, min_size=2)#skip_singletons=True)

# Define Transformation Functions & Apply

In [ ]:
@transformation_function()
def do_nothing(row):
    return row

all_auths = list(by_orgs.keys())

def sample_other_author(auth_label, num_tries=0):
    sample_other_author.tries_collected.append(num_tries)
    
    drawn = all_auths[rand.choice(len(all_auths))]
    
    if num_tries < 5 and drawn == auth_label:
        return sample_other_author(auth_label, num_tries=num_tries+1)
    else:
        return drawn
sample_other_author.tries_collected = []


@transformation_function()
def do_noise(row):
    e_id, auth_label = row["index"], row["org"]
    other_label = sample_other_author(auth_label)
    other_email = rand.choice(by_orgs[other_label])
    
#     new_row = row.copy()
    for n in names:
        row[n+"_2"] = getattr(other_email, n)

    return row

tfs = [do_nothing, do_noise]

In [ ]:
mf_policy = MeanFieldPolicy(
        len(tfs), sequence_length=1, p=[0.3, 0.7], n_per_original=2, keep_original=False
)

tf_applier = PandasTFApplier(tfs, mf_policy)
samples = tf_applier.apply(mail_frame)

# Label Rows

In [ ]:
@labeling_function()
def label(row):
    return row.org == row.org_2

In [ ]:
lf_applier = PandasLFApplier([label])
labelled = lf_applier.apply(samples)

samples["is_same"] = labelled

In [ ]:
samples[["index", "index_2", "is_same"]].shape

# Inspect Results

In [ ]:
same_author = lambda df: df[df.org == df.org_2].shape[0]/df.shape[0]
same_mail =  lambda df: df[df["id"] == df["id_2"]].shape[0]/df.shape[0]
duplicates = lambda df: 1 - df.drop_duplicates().shape[0]/df.shape[0]

In [ ]:
same_author(samples), same_mail(samples), duplicates(samples)

# Split into Train and Test

In [ ]:
samples = pd.read_csv("org_samples.csv.gz")

In [ ]:
shuffled_samples = samples.sample(frac=1)[:100000]

ratio = 0.7
nrows = shuffled_samples.shape[0]

train, test = shuffled_samples[:int(nrows*ratio)], shuffled_samples[int(nrows*ratio):]

In [ ]:
print(same_author(samples), same_author(train), same_author(test))
print(same_mail(samples), same_mail(train), same_mail(test))
print(duplicates(samples), duplicates(train), duplicates(test))

# Save everything

In [ ]:
samples.to_csv("org_samples.csv.gz")

In [ ]:
sample_matrix = samples[["index", "index_2", "is_same"]].to_numpy().astype("int")
with open("org_train_and_test_inds.pkl", "wb") as handle:
    pickle.dump(sample_matrix, handle)

In [ ]:
train_matrix = train[["index", "index_2", "is_same"]].to_numpy().astype("int")
with open("org_train_inds.pkl", "wb") as handle:
    pickle.dump(train_matrix, handle)

In [ ]:
test_matrix = test[["index", "index_2", "is_same"]].to_numpy().astype("int")
with open("org_test_inds.pkl", "wb") as handle:
    pickle.dump(test_matrix, handle)

In [ ]:
senders_str = [str(s) for s in orgs]
with open("orgs.pkl", "wb") as handle:
    pickle.dump(senders_str, handle)